In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp dynamodb

# dynamodb

> Interact with dynamodb.

In [ ]:
#| hide
from fastcore.test import *
from nbdev.showdoc import *

In [ ]:
#| export
from dataclasses import dataclass
from pathlib import Path
from typing import Any, Callable, Dict, Iterable, Optional, Sequence, Tuple

import boto3
from boto3.dynamodb.conditions import Attr
import botocore
from botocore.exceptions import BotoCoreError
import rtime.core as rtime

In [ ]:
#| export
TEN_MINUTES_IN_MS = 1000 * 60 * 10

In [ ]:
#| export
@dataclass
class Config:
    """Configuration for a dynamodb instance."""
    region: str
    aki: str | Path
    sak: str | Path


In [ ]:
#| export
class DynamoDB:
    """Wrapper around dynamodb resource."""
    
    def __init__(self, config: Config):
        self.timestamp = 0
        self.resource = self._init_resource()
        self.config = config

    def _init_resource(self):
        """Return the main table."""
        config = self.config
        with open(config.aki, "r") as infile:
            aki = infile.read().strip()
        with open(config.sak, "r") as infile:
            sak = infile.read().strip()
    
        self.timestamp = rtime.timestamp()
        return boto3.resource(
            "dynamodb",
            region_name=config.region,
            aws_access_key_id=aki,
            aws_secret_access_key=sak,
        )
        
    def __getattr__(self, name):
        """Forward requests to the underlying resource."""
        if rtime.timestamp() - self.timestamp > TEN_MINUTES_IN_MS:
            self.resource = self._init_resource()
        return getattr(self.resource, name)


In [ ]:
#| hide
import nbdev

nbdev.nbdev_export()